# Pinecone Hybrid Search

>[Pinecone](https://docs.pinecone.io/docs/overview) is a vector database with broad functionality.

This notebook goes over how to use a retriever that under the hood uses Pinecone and Hybrid Search.

The logic of this retriever is taken from [this documentation](https://docs.pinecone.io/docs/hybrid-search)

To use Pinecone, you must have an API key and an Environment. 
Here are the [installation instructions](https://docs.pinecone.io/docs/quickstart).

In [ ]:
#!pip install pinecone-client pinecone-text


In [1]:
import os
import getpass

os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")


Pinecone API Key: ········


In [3]:
from langchain.retrievers import PineconeHybridSearchRetriever
from langchain.retrievers.pinecone_hybrid_search import PineconeIndexUpsert


In [4]:
os.environ["PINECONE_ENVIRONMENT"] = getpass.getpass("Pinecone Environment:")


Pinecone Environment: ········


We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


OpenAI API Key: ········


## Setup Pinecone

You should only have to do this part once.

Note: it's important to make sure that the "context" field that holds the document text in the metadata is not indexed. Currently you need to specify explicitly the fields you do want to index. For more information checkout Pinecone's [docs](https://docs.pinecone.io/docs/manage-indexes#selective-metadata-indexing).

In [6]:
import os
import pinecone

api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
# find environment next to your API key in the Pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

index_name = "langchain-pinecone-hybrid-search"

pinecone.init(api_key=api_key, environment=env)
pinecone.whoami()


WhoAmIResponse(username='c78f2bd', user_label='default', projectname='5c4ca0b')

In [77]:
# create the index
pinecone.create_index(
    name=index_name,
    dimension=1536,  # dimensionality of dense model
    metric="dotproduct",  # sparse values supported only for dotproduct
    pod_type="s1",
    metadata_config={"indexed": []},  # see explanation above
)

Now that its created, we can use it.
We can create a pinecone index by using our wrapper for upsert so we either choose between multithreaded and singlethreaded implementation.
Pass `pool_threads=4` if you want to use multithreaded implementation of the upsert.

In [9]:
index = PineconeIndexUpsert.get_index_upsert(index_name, pool_threads=1)


## Get embeddings and sparse encoders

Embeddings are used for the dense vectors, tokenizer is used for the sparse vector

In [10]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


To encode the text to sparse values you can either choose SPLADE or BM25. For out of domain tasks we recommend using BM25.

For more information about the sparse encoders you can checkout pinecone-text library [docs](https://pinecone-io.github.io/pinecone-text/pinecone_text.html).

In [11]:
from pinecone_text.sparse import BM25Encoder

# or from pinecone_text.sparse import SpladeEncoder if you wish to work with SPLADE

# use default tf-idf values
bm25_encoder = BM25Encoder().default()


100% [........................................................................] 65406227 / 65406227

The above code is using default tfids values. It's highly recommended to fit the tf-idf values to your own corpus. You can do it as follow:

```python
corpus = ["foo", "bar", "world", "hello"]

# fit tf-idf values on your corpus
bm25_encoder.fit(corpus)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")
```

## Load Retriever

We can now construct the retriever!

In [13]:
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings, sparse_encoder=bm25_encoder, index_upsert=index
)


## Add texts (if necessary)

We can optionally add texts to the retriever (if they aren't already in there)

In [14]:
retriever.add_texts(["foo", "bar", "world", "hello"])


  0%|          | 0/1 [00:00<?, ?it/s]

Upserted vectors:   0%|          | 0/4 [00:00<?, ?it/s]

## Use Retriever

We can now use the retriever!

In [15]:
result = retriever.get_relevant_documents("foo")


In [16]:
result[0]


Document(page_content='foo')